In [1]:
from dataclasses import dataclass
from datetime import datetime, timedelta
import os
from pathlib import Path
from typing import List
pwd = Path(os.getcwd())

import pytz
import random
import sys
sys.path.append(str(pwd.parent))
sys.path.append(str(pwd.parent / "config"))

import warnings
warnings.simplefilter("ignore")

import networkx as nx
import numpy as np
from osmread import parse_file, Node, Way
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

from config import Config
from util import haversine
import matplotlib.pyplot as plt
import matplotlib as mpl

config = Config.load()

In [2]:
time_step = int(config.TIMESTEP) / 1e9 / 60

In [3]:
import glob
import json
adj_data_path = glob.glob(f"../data/{config.DATA_SIZE}/*Neighbor.csv")[0]
size = len(pd.read_csv(adj_data_path, header=None))
A = np.zeros((size,size))
with open(adj_data_path) as f:
    for i in range(size):
        tmp = f.readline().replace('""','"').replace('"{','{').replace('}"','}').rstrip()
        for info_str in tmp.split("},"):
            if info_str[-1] != "}":
                info_str += "}"
            info = json.loads(info_str)
            if info["distance"] < time_step:
                A[i][info["area_id"]] = 1
                A[info["area_id"]][i] = 1

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self, num_node_features: int, num_nodes):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_node_features, num_nodes)
        self.conv2 = GCNConv(num_nodes, num_nodes)
        self.fc = torch.nn.Linear(num_nodes*num_nodes, num_nodes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = torch.flatten(x)
        return self.fc(x)
        

In [5]:
src_list = []
dst_list = []
for i in range(size):
    for j in range(size):
        if A[i][j] == 1:
            src_list.append(i)
            dst_list.append(j)
edge_index = torch.tensor([src_list, dst_list], dtype=torch.long)

# create training data

In [6]:
node_data_df = pd.read_csv(f"../data/{config.DATA_SIZE}/NodeConnection.csv")[["NodeID", "Latitude", "Longitude"]].drop_duplicates()

In [7]:
order_df = pd.read_csv(f"../data/yellow_tripdata_2015-06.csv")

USE_COLUMNS = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
]
RENAME_DICT = {
    "tpep_pickup_datetime": "Start_time",
    "tpep_dropoff_datetime": "End_time",
    "pickup_longitude": "PointS_Longitude",
    "pickup_latitude": "PointS_Latitude",
    "dropoff_longitude": "PointE_Longitude",
    "dropoff_latitude": "PointE_Latitude",
}

rename_order_df = order_df[USE_COLUMNS].rename(columns=RENAME_DICT).sort_values("Start_time").reset_index(drop=True)

# NewYorkエリア外のレコードを除去する.
rename_order_without_outlier_df = rename_order_df[
    (rename_order_df["PointS_Longitude"] > config.LOCAL_REGION_BOUND.west_bound)
    & (rename_order_df["PointS_Longitude"] < config.LOCAL_REGION_BOUND.east_bound)
    & (rename_order_df["PointE_Longitude"] > config.LOCAL_REGION_BOUND.west_bound)
    & (rename_order_df["PointE_Longitude"] < config.LOCAL_REGION_BOUND.east_bound)
    & (rename_order_df["PointS_Latitude"] > config.LOCAL_REGION_BOUND.south_bound)
    & (rename_order_df["PointS_Latitude"] < config.LOCAL_REGION_BOUND.north_bound)
    & (rename_order_df["PointE_Latitude"] > config.LOCAL_REGION_BOUND.south_bound)
    & (rename_order_df["PointE_Latitude"] < config.LOCAL_REGION_BOUND.north_bound)
]

In [8]:
nn = NearestNeighbors(algorithm='ball_tree')
nn.fit(node_data_df[["Longitude", "Latitude"]].values)

node_id_list = node_data_df["NodeID"].values.tolist()

_, start_indices = nn.kneighbors(
    rename_order_without_outlier_df[["PointS_Longitude", "PointS_Latitude"]], n_neighbors=1)
rename_order_without_outlier_df["NodeS"] = [node_id_list[i[0]] for i in start_indices]

In [9]:
node_to_grid_df = pd.read_csv(f"../data/{config.DATA_SIZE}/(-74.0088, -73.9778, 40.7111, 40.7471)53TransportationClusteringCluster.csv")
node_to_grid_info = {}
for node_id, grid_id in zip(node_to_grid_df["NodeID"], node_to_grid_df["GridID"]):
    node_to_grid_info[node_id] = grid_id

In [10]:
rename_order_without_outlier_df.head()

,Start_time,End_time,PointS_Longitude,PointS_Latitude,PointE_Longitude,PointE_Latitude,NodeS
30,2015-06-01 00:00:08,2015-06-01 00:36:25,-73.987793,40.732361,-73.987793,40.732361,8309479665
34,2015-06-01 00:00:09,2015-06-01 00:01:52,-74.005989,40.735191,-73.999718,40.743622,8307463843
35,2015-06-01 00:00:09,2015-06-01 00:02:49,-73.988045,40.731632,-73.980858,40.730545,8309479665
45,2015-06-01 00:00:14,2015-06-01 00:08:42,-73.982330,40.744888,-74.003281,40.735100,8119462495
54,2015-06-01 00:00:16,2015-06-01 00:08:59,-73.991737,40.729820,-73.984352,40.716393,5706569384


In [11]:
rename_order_without_outlier_df["GridID"] = rename_order_without_outlier_df["NodeS"].map(node_to_grid_info)
rename_order_without_outlier_df.head()

,Start_time,End_time,PointS_Longitude,PointS_Latitude,PointE_Longitude,PointE_Latitude,NodeS,GridID
30,2015-06-01 00:00:08,2015-06-01 00:36:25,-73.987793,40.732361,-73.987793,40.732361,8309479665,37
34,2015-06-01 00:00:09,2015-06-01 00:01:52,-74.005989,40.735191,-73.999718,40.743622,8307463843,40
35,2015-06-01 00:00:09,2015-06-01 00:02:49,-73.988045,40.731632,-73.980858,40.730545,8309479665,37
45,2015-06-01 00:00:14,2015-06-01 00:08:42,-73.982330,40.744888,-74.003281,40.735100,8119462495,52
54,2015-06-01 00:00:16,2015-06-01 00:08:59,-73.991737,40.729820,-73.984352,40.716393,5706569384,30


In [12]:
def newyork_datetime_to_utc(datetime_str: str) -> int:
    eastern = pytz.timezone('US/Eastern')
    date = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    date.astimezone(eastern)
    return int(date.timestamp())

rename_order_without_outlier_df["Start_datetime"] = rename_order_without_outlier_df["Start_time"].apply(
    lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
)
rename_order_without_outlier_df["month"] = rename_order_without_outlier_df["Start_datetime"].apply(lambda x: x.month)
rename_order_without_outlier_df["day"] = rename_order_without_outlier_df["Start_datetime"].apply(lambda x: x.day)
rename_order_without_outlier_df["hour"] = rename_order_without_outlier_df["Start_datetime"].apply(lambda x: x.hour)
rename_order_without_outlier_df["minute"] = rename_order_without_outlier_df["Start_datetime"].apply(lambda x: (((x.minute // 10)) % 6) * 10 )

In [13]:
months = [6]
days = list(range(1,32))
hours = list(range(0,24))
minutes = list(range(0,60, 10))

In [14]:
from sklearn.preprocessing import StandardScaler

cols = ["GridID", "month", "day", "hour", "minute"]
raw_feature_df = rename_order_without_outlier_df[cols]

date_feature = []
for grid_id in sorted(raw_feature_df["GridID"].unique()):
    tmp1 = raw_feature_df[(raw_feature_df["GridID"]==grid_id)]
    for month in months:
        tmp2 = tmp1[(tmp1["month"]==month)]
        before = 0
        for day in days:
            tmp3 = tmp2[(tmp2["day"]==day)]
            for hour in hours:
                tmp4 = tmp3[(tmp3["hour"]==hour)]
                for minute in minutes:
                    tmp5 = tmp4[(tmp4["minute"]==minute)]
                    date_feature.append([grid_id, month, day, hour, minute, len(tmp5), before])
                    before = len(tmp5)
full_feature_df = pd.DataFrame(date_feature, columns=["GridID", "month", "day", "hour", "minute", "target", "before"])

feature_columns = ["GridID", "month", "day", "hour", "minute", "before"]
ss = StandardScaler()
features = ss.fit_transform(full_feature_df[feature_columns].values)

full_feature_df.rename(columns={col: "raw_" + col for col in feature_columns}, inplace=True)
scaled_feature = pd.DataFrame(features, columns=feature_columns)
feature_df = pd.concat([full_feature_df, scaled_feature], axis=1)

In [15]:
class Data:
    feature: list
    target: list

class DatasetIterator:
    def __init__(self, features, targets, batch_size=64):
        self.__batch_size = batch_size
        self.__features = features
        self.__targets = targets
        self.__i = 0
        self.__numbers = 0
        self.__random_index = list(range(len(self.__features)))
        random.shuffle(self.__random_index)

    @property
    def num_batch(self):
        return len(self.__features) // self.__batch_size

    def __iter__(self):
        return self

    def __next__(self):
        if self.__i == self.num_batch:
            idx_list = self.__random_index[self.__i * self.__batch_size:]
            self.__i += 1
            return Data(
                feature=self.__features[idx_list],
                target=self.__targets[idx_list],
            )
        if self.__i == (self.num_batch + 1):
            raise StopIteration()
        idx_list = self.__random_index[self.__i * self.__batch_size:(self.__i + 1) * self.__batch_size]
        self.__i += 1
        return Data(
                feature=self.__features[idx_list],
                target=self.__targets[idx_list],
            )
    

class Dataset:
    def __init__(self, batch_size=64):
        self.__batch_size = batch_size
        self.__features = []
        self.__targets = []

    def add_feature(self, feature):
        self.__features.append(feature)

    def add_target(self, target):
        self.__targets.append(target)

    @property
    def num_batch(self):
        return len(self.__features) // self.__batch_size
        
    def __iter__(self):
        return DatasetIterator(np.array(self.__features), np.array(self.__targets.copy()), self.__batch_size)

In [16]:
feature_df.head()

,raw_GridID,raw_month,raw_day,raw_hour,raw_minute,target,raw_before,GridID,month,day,hour,minute,before
0,0,6,1,0,0,3,0,-1.699673,0.0,-1.677051,-1.661325,-1.46385,-0.630654
1,0,6,1,0,10,0,3,-1.699673,0.0,-1.677051,-1.661325,-0.87831,-0.321227
2,0,6,1,0,20,0,0,-1.699673,0.0,-1.677051,-1.661325,-0.29277,-0.630654
3,0,6,1,0,30,0,0,-1.699673,0.0,-1.677051,-1.661325,0.29277,-0.630654
4,0,6,1,0,40,1,0,-1.699673,0.0,-1.677051,-1.661325,0.87831,-0.630654


In [17]:
from torch_geometric.data import Data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = Net(num_node_features=len(feature_columns), num_nodes=size).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)

dataset = Dataset()

for month in months:
    for day in days:
        for hour in hours:
            for minute in minutes:
                dataset.add_feature(
                    feature_df[
                        (feature_df["raw_month"]==month)
                        & (feature_df["raw_day"]==day)
                        & (feature_df["raw_hour"]==hour)
                        & (feature_df["raw_minute"]==minute)
                    ][feature_columns].values
                )
                dataset.add_target(
                    feature_df[
                        (feature_df["raw_month"]==month)
                        & (feature_df["raw_day"]==day)
                        & (feature_df["raw_hour"]==hour)
                        & (feature_df["raw_minute"]==minute)
                    ]["target"].values
                )


In [26]:
net.train()
losses = []
for _ in range(10):
    for batch in dataset:
        targets = None
        preds = None
        optimizer.zero_grad()
        for feature, target in zip(batch.feature, batch.target):
            feature_tensor = torch.tensor(feature, dtype=torch.float)
            y_tensor = torch.tensor(target, dtype=torch.float)
            data = Data(x=feature_tensor, edge_index=edge_index, y=y_tensor)
            pred = net(data)
            if preds is None:
                targets = y_tensor
                preds = pred
            else:
                targets = torch.concat([targets, y_tensor])
                preds = torch.concat([preds, pred], axis=1)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach())

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [29]:
print(preds[:20])
print(targets[:20])

tensor([ 5.5776,  0.2739,  0.3792,  0.0576,  0.2776, -0.0718,  0.0447,  3.1127,
         3.4319,  3.8104,  1.3884,  1.4837,  0.7562, -0.1270,  3.3952,  6.1816,
         5.6817,  3.9005,  5.6707,  1.3940], grad_fn=<SliceBackward0>)
tensor([5., 1., 2., 0., 0., 0., 0., 3., 1., 2., 3., 2., 1., 0., 6., 7., 7., 4.,
        5., 0.])


In [24]:
pred

tensor([6.1920e+00, 1.9017e-01, 5.5403e-01, 3.3749e-02, 2.5348e-01, 1.2474e-01,
        1.4732e-01, 3.2242e+00, 3.2069e+00, 3.2018e+00, 1.9371e+00, 1.8312e+00,
        3.3564e-01, 7.8776e-02, 4.4853e+00, 6.9854e+00, 4.9925e+00, 4.0602e+00,
        6.1732e+00, 7.2308e-01, 7.8857e+00, 1.0326e+01, 1.5583e+01, 8.5112e+00,
        3.6915e+00, 6.5833e+00, 9.7948e-01, 2.9579e+00, 6.2705e+00, 2.6912e+00,
        2.5921e+01, 4.2624e+00, 2.6120e+00, 7.4406e+00, 6.7321e+00, 1.1416e+01,
        6.1600e+00, 1.0879e+01, 4.3430e+00, 1.9364e+00, 4.4275e+00, 2.6820e+00,
        4.1005e+00, 8.0188e+00, 4.6335e+01, 6.6539e+00, 2.9959e+01, 8.9711e+00,
        3.8669e+00, 7.6224e+00, 3.2217e+00, 4.0208e+01, 9.8994e+00],
       grad_fn=<AddBackward0>)